In [19]:
import torch
from torch.utils.data import DataLoader

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import evaluate

## GPT Model

### Pipeline Usage

In [ ]:
gpt = pipeline('text-generation', model='openai-gpt')

In [ ]:
prompt = "Once upon a time, there was a mad king"
answer = gpt(prompt, max_length=30)
print(answer[0]["generated_text"])

### Importing the model as Classifier and the tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai-gpt")
model = AutoModelForSequenceClassification.from_pretrained("openai-gpt")

Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Adding a pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad_token_id in the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# Make sure the model is aware of the new tokens
model.resize_token_embeddings(len(tokenizer))

Embedding(40479, 768)

In [4]:
inputs = tokenizer("Hello, I'm testing the first gpt", return_tensors="pt")
inputs

{'input_ids': tensor([[3570,  240,  249,  256,  258, 8448,  481,  929,   30, 1992]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
inputs = tokenizer("Hello, I'm testing the first gpt", return_tensors="pt")
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3745,  0.1116]], grad_fn=<IndexBackward0>), hidden_states=None, attentions=None)

In [6]:
prediction = torch.argmax(outputs.logits)
prediction

tensor(1)

## SST-2 Dataset

In [3]:
dataset = load_dataset("sst2")
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [4]:
print(dataset["train"]["sentence"][0])
print(dataset["train"]["label"][0])

hide new secretions from the parental units 
0


In [12]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [13]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1_000))
small_eval_dataset = tokenized_datasets["validation"]

In [14]:
small_train_dataset

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [15]:
small_eval_dataset

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 872
})

## Finetuning

In [13]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  num_train_epochs=2,
                                  learning_rate=1e-3,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch")

In [14]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 0.7535799145698547, 'eval_accuracy': 0.5091743119266054, 'eval_runtime': 55.1496, 'eval_samples_per_second': 15.812, 'eval_steps_per_second': 1.976, 'epoch': 1.0}


  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 0.6989779472351074, 'eval_accuracy': 0.5091743119266054, 'eval_runtime': 53.4718, 'eval_samples_per_second': 16.308, 'eval_steps_per_second': 2.038, 'epoch': 2.0}
{'train_runtime': 1222.9312, 'train_samples_per_second': 1.635, 'train_steps_per_second': 0.204, 'train_loss': 0.7675388793945312, 'epoch': 2.0}


TrainOutput(global_step=250, training_loss=0.7675388793945312, metrics={'train_runtime': 1222.9312, 'train_samples_per_second': 1.635, 'train_steps_per_second': 0.204, 'train_loss': 0.7675388793945312, 'epoch': 2.0})

In [17]:
trainer.save_model("gpt_sentiment_classifier")

## Evaluating the Fine-tuned Model

In [19]:
trainer.evaluate()

  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 0.6989779472351074,
 'eval_accuracy': 0.5091743119266054,
 'eval_runtime': 55.0965,
 'eval_samples_per_second': 15.827,
 'eval_steps_per_second': 1.978,
 'epoch': 2.0}

In [27]:
np.sum(small_eval_dataset["label"]) / len(small_eval_dataset["label"])

0.5091743119266054

In [20]:
trainer.model

OpenAIGPTForSequenceClassification(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40479, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

## Using the Saved Model

In [30]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained("./gpt_sentiment_classifier/")
tokenizer = AutoTokenizer.from_pretrained("openai-gpt")

In [31]:
# Adding a pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [32]:
finetuned_model

OpenAIGPTForSequenceClassification(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40479, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [33]:
small_eval_dataset

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 872
})

In [34]:
# Convert dataset to PyTorch tensors
input_ids = torch.tensor(small_eval_dataset['input_ids'])
attention_mask = torch.tensor(small_eval_dataset['attention_mask'])
labels = torch.tensor(small_eval_dataset['label'])

In [38]:
# Combine inputs and labels into a TensorDataset
eval_dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, labels)

# Set batch size
batch_size = 32

# Create a DataLoader
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

In [39]:
# Make predictions in batches
finetuned_model.eval()
all_predictions = []

with torch.no_grad():
    for batch in tqdm(eval_dataloader):
        batch_input_ids, batch_attention_mask, batch_labels = batch

        # Make predictions for the current batch
        batch_outputs = finetuned_model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        batch_predictions = batch_outputs.logits.argmax(dim=1)

        all_predictions.extend(batch_predictions.cpu().numpy())

  0%|          | 0/28 [00:00<?, ?it/s]

In [41]:
len(all_predictions)

872

In [42]:
# All preds are 1?
np.mean(all_predictions)

1.0

In [43]:
# Convert predictions to numpy array
all_predictions = np.array(all_predictions)

# Calculate accuracy
accuracy = (all_predictions == labels.cpu().numpy()).mean()

print("Accuracy:", accuracy)

Accuracy: 0.5091743119266054
